### 기각

##### 기각 사유
선택한 단어가 각 줄거리에서 얼마나 자주 나타나는지와 자주나타나지않는지의 가중치를 구하여 각 가중치를 곱하는 방식하여 유사도를 측정하는 방식인데
선택한 단어들 마다 유사도가 너무 낮아 줄거리를 다시 확인해본 결과 줄거리 자체가 짧다보니 반복되는 단어를 최대한 줄이는 형식으로 인해 단어의 빈도수로 측정하기 어려움을 확인 

In [7]:
import pandas as pd
import numpy as np
import re #  Python의 정규 표현식(regular expression) 모듈 -> re.sub() 함수를 사용하여 문자열에서 정규 표현식 패턴에 일치하는 부분을 다른 문자열로 대체
import html # html 엔터티 코드 제거를 위해 사용
from sklearn.feature_extraction.text import TfidfVectorizer # sklearn 안에 있는 패키지
from sklearn.metrics.pairwise import cosine_similarity # TF-IDF 값을 기준으로 코사인 유사도를 통해 가장 유사한 제목을 찾아주기떄문에 cosine_similarity를 사용

#### 데이터 전처리 (특수문자 제거 및 null값 처리 후 csv파일로 저장)

In [8]:
# 데이터를 데이터프레임으로 로드
data = pd.read_csv("../library_book_data.csv", encoding='utf-8')# 데이터 파일 경로
print(data)

FileNotFoundError: [Errno 2] No such file or directory: '../libray_book_data.csv'

In [ ]:
# # Null 값은 빈 문자열로 대체
# data.fillna("", inplace=True)
# print(data)

/tmp/ipykernel_8124/3805302243.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.fillna("", inplace=True)


         SEQ_NO  ISBN_THIRTEEN_NO VLM_NM  \
0       6352228     9791156759270          
1       6352229     9791168120877          
2       6352230     9791168120839          
3       6352231     9791168120846          
4       6352232     9791168120747          
...         ...               ...    ...   
157113  6372497     9781191496148          
157114  6372498     9781161755015          
157115  6372499     9781197192907          
157116  6372500     9781188005384          
157117  6372501     9781191334426          

                                                 TITLE_NM  \
0            너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이   
1                   일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이   
2                             본격 한중일 세계사 12 - 임오군란과 통킹 위기   
3                    즉시 기분을 바꿔드립니다 - 신기하게 마음이 편해지는 응급 처방전   
4                               오늘도 리추얼 : 음악, 나에게 선물하는 시간   
...                                                   ...   
157113            설민석의 세계사 대모험 11 (영국 산업 혁명 

In [ ]:
# # 모든 열에 대해 특수 문자와 HTML 엔터티 코드 모두 제거
# for col in data.columns:
#     data[col] = data[col].apply(lambda x: re.sub(r'[^\w\s]', '', html.unescape(str(x))))
# # 결과 출력
# print(data)

         SEQ_NO ISBN_THIRTEEN_NO VLM_NM  \
0       6352228    9791156759270          
1       6352229    9791168120877          
2       6352230    9791168120839          
3       6352231    9791168120846          
4       6352232    9791168120747          
...         ...              ...    ...   
157113  6372497    9781191496148          
157114  6372498    9781161755015          
157115  6372499    9781197192907          
157116  6372500    9781188005384          
157117  6372501    9781191334426          

                                                 TITLE_NM  \
0                     너에게 목소리를 보낼게  달빛천사 성우 이용신의 첫 번째 에세이   
1                    일기에도 거짓말을 쓰는 사람  99년생 시인의 자의식 과잉 에세이   
2                              본격 한중일 세계사 12  임오군란과 통킹 위기   
3                     즉시 기분을 바꿔드립니다  신기하게 마음이 편해지는 응급 처방전   
4                                 오늘도 리추얼  음악 나에게 선물하는 시간   
...                                                   ...   
157113               설민석의 세계사 대모험 11 영국 산업 혁명 편 혁명에 맞선 자

In [ ]:
# data.to_csv('../cleaned_data.csv', index=False, encoding='utf-8')

#### TF-IDF
사용해 텍스트 문서에서 각 단어의 중요성을 가중치로 나타내어 각 단어의 중요성을 나타내고, 주어진 문서 내에서 해당 단어가 얼마나 중요한지를 측정 (빈도수)

In [ ]:
# 데이터를 데이터프레임으로 로드
data = pd.read_csv('../cleaned_data.csv', encoding='utf-8')# 데이터 파일 경로
print(data)

       SEQ_NO  ISBN_THIRTEEN_NO VLM_NM                              TITLE_NM  \
0     6352228     9791156759270    NaN   너에게 목소리를 보낼게  달빛천사 성우 이용신의 첫 번째 에세이   
1     6352229     9791168120877    NaN  일기에도 거짓말을 쓰는 사람  99년생 시인의 자의식 과잉 에세이   
2     6352230     9791168120839    NaN            본격 한중일 세계사 12  임오군란과 통킹 위기   
3     6352231     9791168120846    NaN   즉시 기분을 바꿔드립니다  신기하게 마음이 편해지는 응급 처방전   
4     6352232     9791168120747    NaN               오늘도 리추얼  음악 나에게 선물하는 시간   
...       ...               ...    ...                                   ...   
1997  6354246     9791164670871    NaN                          학교 서클대화가 필요해   
1998  6354247     9788994229003    NaN                      People Make City   
1999  6354248     9788994027203    NaN                                 동방해경표   
2000  6354249     9791156759126    NaN                      큰글자도서 그렇다면 정상입니다   
2001  6354250     9788988963258    NaN                                 그 사람들   

                  AUTHR_NM PUBLISHER_NM

In [ ]:
data["BOOK_INTRCN_CN"].fillna("NaN", inplace=True)

In [ ]:
# TF-IDF 벡터화기 생성
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
# TF-IDF 매트릭스 계산
tfidf_matrix = tfidf_vectorizer.fit_transform(data["BOOK_INTRCN_CN"])

In [ ]:
print(tfidf_matrix) 

  (0, 4856)	0.07956134293673803
  (0, 6217)	0.1480195298363794
  (0, 3027)	0.17189629176721588
  (0, 5059)	0.17189629176721588
  (0, 6121)	0.1630840893574864
  (0, 2330)	0.17189629176721588
  (0, 4100)	0.14176717272500183
  (0, 3776)	0.17189629176721588
  (0, 5268)	0.17189629176721588
  (0, 4940)	0.17189629176721588
  (0, 3708)	0.14466928596811335
  (0, 4239)	0.1519820272991616
  (0, 3114)	0.12538502402599394
  (0, 4694)	0.34379258353443176
  (0, 3584)	0.34379258353443176
  (0, 3531)	0.17189629176721588
  (0, 5580)	0.17189629176721588
  (0, 1374)	0.1267026132038948
  (0, 3159)	0.17189629176721588
  (0, 228)	0.17189629176721588
  (0, 2735)	0.34379258353443176
  (0, 4291)	0.17189629176721588
  (0, 2598)	0.17189629176721588
  (0, 5270)	0.1480195298363794
  (0, 2267)	0.17189629176721588
  :	:
  (2000, 6075)	0.19216328182169165
  (2000, 1790)	0.19216328182169165
  (2000, 5401)	0.19216328182169165
  (2000, 3485)	0.19216328182169165
  (2000, 3335)	0.19216328182169165
  (2000, 4340)	0.19216328

In [ ]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
print(cosine_sim)

[[1.        0.0145301 0.        ... 0.        0.        0.       ]
 [0.0145301 1.        0.        ... 0.        0.        0.       ]
 [0.        0.        1.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 1.        0.        1.       ]
 [0.        0.        0.        ... 0.        1.        0.       ]
 [0.        0.        0.        ... 1.        0.        1.       ]]


In [ ]:
# 선택한 줄거리
selected_document = "음악"

In [ ]:
# TF-IDF 매트릭스에 주어진 줄거리를 추가
# 선택한 단어(또는 문서)를 TF-IDF 가중치 벡터로 변환하여 저장
tfidf_matrix_selected = tfidf_vectorizer.transform([selected_document])

In [ ]:
# 주어진 줄거리와 다른 모든 도서의 유사도 계산
# 선택한 단어(또는 문서)와 다른 모든 단어(또는 문서) 간의 코사인 유사도
cosine_sim_selected = cosine_similarity(tfidf_matrix_selected, tfidf_matrix)
similar_books = list(enumerate(cosine_sim_selected[0]))

In [ ]:
# 가장 유사한 N개의 도서 추천
# similar_books를 코사인 유사도 값에 따라 내림차순으로 정렬 -> 상위 N개의 유사한 단어를 선택하는 과정
N = 5
similar_books.sort(key=lambda x: x[1], reverse=True) # key 매개변수에는 x[1]을 사용하여 각 쌍(단어 인덱스, 유사도)에서 두 번째 요소, 즉 유사도 값을 기준으로 정렬
recommended_books = similar_books[1:N + 1] # similar_books[1:N + 1]로 첫 번째 요소를 제외한 다음부터 N개까지의 요소를 선택 - > 첫번째는  similar_books 리스트 내에서 선택한 단어(또는 문서) 자체와의 유사도가 항상 1.0이 되기 때문

In [ ]:
# 선택한 줄거리 출력
print(f"선택한 줄거리: {selected_document}")
# 추천된 도서 출력
for i, score in recommended_books:
    print(f"제목: {data.loc[i, 'TITLE_NM']}")
    print(f"줄거리: {data.loc[i, 'BOOK_INTRCN_CN']}")
    print(f"유사도: {score:.2f}")
    print()

선택한 줄거리: 음악
제목: 너에게 목소리를 보낼게  달빛천사 성우 이용신의 첫 번째 에세이
줄거리: 2004년 방영한 애니메이션 달빛천사에서 주인공 루나풀문 역을 맡으며 90년대생들에게 보석 같은 추억을 선물한 성우 이용신의 첫 번째 에세이 수많은 작품의 주연을 맡으며 쉬지 않고 대중에게 행복을 전해온 성우 이용신의 발자취를 확인할 수 있다
유사도: 0.00

제목: 일기에도 거짓말을 쓰는 사람  99년생 시인의 자의식 과잉 에세이
줄거리: 그러니 나는 말하고 싶은 것을 말하겠다침착하게 사랑하기 차도하 시인 첫 에세이새롭고 도발적인 작품성으로 문단의 기대주로 떠오른 차도하 시인의 첫 번째 산문집이 출간됐다 혼자 보는 일기에도 거짓말을 쓸 수밖에 없었던 시인의 산문집 일기에도 거짓말을 쓰는 사람은 공교롭게도 보는 
유사도: 0.00

제목: 본격 한중일 세계사 12  임오군란과 통킹 위기
줄거리: 한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다 러시아의 세력 확장을 경계한 청은 친중결일연미라는 계책을 앞세워 조선에 대한 통제를 강화하고 이를 그냥 보아 넘길 수 없던 일본은 임오군란을 계기로 마수를 뻗치는데 정국 안정화와 근대 문물 도입을 향해 뛰어가는 조선의 앞에 놓인 거대한 장애물을 살펴본다
유사도: 0.00

제목: 즉시 기분을 바꿔드립니다  신기하게 마음이 편해지는 응급 처방전
줄거리: 누구에게나 기분 구급상자가 필요하다 하나씩 하나씩 차근차근 좋은 기분을 쌓고 건강한 마음을 만드는 법을 담은 책이다
유사도: 0.00

제목: 지리산 1  김은희 대본집
줄거리: 국립공원 최고의 레인저 서이강과 말 못 할 비밀을 가진 신입 레인저 강현조가 산에서 일어나는 의문의 사고를 파헤치며 벌어지는 이야기를 그린 지리산의 작가판 대본집이 출간된다 tvN 15주년 기념으로 기획된 이 드라마는 5년 7개월 만에 TV로 돌아온 김은희 작가의 복귀작으로 공개되자마자 두 자릿수 시청률을 기록하는 등 큰 화제를 모으고 있다
유사도: 0.00

